In [1]:
import re
from dataclasses import dataclass


_parse_line = re.compile(r'#\d+\s*@\s*(\d+),(\d+):\s*(\d+)x(\d+)').search

                  
@dataclass
class Rectangle:
    left: int
    top: int
    width: int
    height: int
    
    @classmethod
    def from_line(cls, line: str):
        left, top, width, height = map(int, _parse_line(line).groups())
        return cls(left, top, width, height)

    @property
    def right(self):
        return self.left + self.width

    def at_x(self, x):
        if self.left <= x < self.left + self.width:
            return (self.top, self.top + self.height)

In [2]:
testrect = Rectangle.from_line("#123 @ 3,2: 5x4\n")
assert (testrect.left, testrect.top, testrect.width, testrect.height) == (3, 2, 5, 4)
assert testrect.right == 8
assert all(testrect.at_x(x) is None for x in range(3))
assert all(testrect.at_x(x) == (2, 6) for x in range(3, 8))

This is basically an interval merging problem.

* At each line `x`, find all the rectangles that intersect that line, as intervals (top, bottom)
* Count how many `y` coordinates overlap.
* Take into account that 1 large interval could overlap multiple following intervals that themselves do not overlap.

You can do this by taking the minimum of (furthest bottom, current bottom) minus the current top value, provided top < previous bottom edge:

```
 0
 1 top
 2  |
 3  |  top  X  5 - 3 == 2
 4  |   |   X  5 is the min(furthest bottom, bot) value here
 5 bot  |
 6      |
 7     bot
 8 
 9 top
10  |
11  |  top X  13 - 11 == 2
12  |   |  X  13 is the min(furthest bottom, bot) value here
13  |  bot
14  |
15  |  top X  17 - 15 == 2
16  |   |  X  17 is the min(furthest bottom, bot) value here
17  |  bot
18 bot
```

 Gotcha is multiple rectangles overlapping on the same `y` coordinates overlap:

```
 0
 1 top
 2  |
 3  |  top     X
 4  |   |  top X <- only one overlapping square, don't count this twice
 5 bot  |   |  X
 6      |   |  X total 4
 7     bot  |
 8          |
 9         bot
```

So we track the furthest overlap already handled by storing the previous bottom value, to prefer over the current top value. In the above, we already handled the (5 - 3) overlap, so the next overlap to calculate should use 5 rather than 4 for the `top` value.

Another edgecase is where a short interval follows an already overlapping longer interval; the bottom of the shorter interval is then above the already covered overlap bottom:

```
 0
 1 top
 2  |
 3  |  top     X
 4  |   |  top X The short interval can be ignored here
 5  |   |   |  X
 6  |   |  bot X
 7  |   |      X total 5  
 8 bot  |       
 9     bot     
```

In [3]:
from operator import methodcaller

def sum_overlaps(rectangles):
    max_x = max(r.right for r in rectangles)
    total = 0
    for x in range(max_x):
        intervals = sorted(filter(None, map(methodcaller("at_x", x), rectangles)))
        overlap = overlap_bottom = furthest_bottom = 0
        for top, bottom in intervals:
            if top < furthest_bottom and bottom > overlap_bottom:
                overlap += min(furthest_bottom, bottom) - max(top, overlap_bottom)
                overlap_bottom = min(furthest_bottom, bottom)
            furthest_bottom = max(furthest_bottom, bottom)
        total += overlap
    return total

In [4]:
test1 = [Rectangle.from_line(l) for l in """\
#1 @ 1,3: 4x4
#2 @ 3,1: 4x4
#3 @ 5,5: 2x2
""".splitlines()]
assert sum_overlaps(test1) == 4
test2 = [Rectangle.from_line(l) for l in """\
#1 @ 1,9: 1x9
#2 @ 1,11: 1x2
#3 @ 1,15: 1x2
""".splitlines()]
assert sum_overlaps(test2) == 4
test3 = [Rectangle.from_line(l) for l in """\
#1 @ 1,1: 1x4
#2 @ 1,3: 1x4
#3 @ 1,4: 1x5
""".splitlines()]
assert sum_overlaps(test3) == 4
test3 = [Rectangle.from_line(l) for l in """\
#1 @ 1,1: 1x7
#2 @ 1,3: 1x6
#3 @ 1,4: 1x2
""".splitlines()]
assert sum_overlaps(test3) == 5

In [5]:
import aocd
data = aocd.get_data(day=3, year=2018)
rectangles = [Rectangle.from_line(l) for l in data.splitlines()]

In [6]:
print("Part 1:", sum_overlaps(rectangles))

Part 1: 106501


In [7]:
from operator import methodcaller

def no_overlaps(rectangles):
    max_x = max(r.right for r in rectangles)
    no_overlaps = set(range(len(rectangles)))
    for x in range(max_x):
        interval_and_id = ((r.at_x(x), i) for i, r in enumerate(rectangles))
        interval_and_id = sorted((interval, id_) for interval, id_ in interval_and_id if interval)
        overlap = overlap_bottom = furthest_bottom = 0
        id_of_furthest_bottom = None
        for (top, bottom), id_ in interval_and_id:
            if top < furthest_bottom:
                no_overlaps.discard(id_)
                no_overlaps.discard(id_of_furthest_bottom)
                overlap_bottom = min(furthest_bottom, bottom)
            if bottom > furthest_bottom:
                furthest_bottom = bottom
                id_of_furthest_bottom = id_
    return next(iter(no_overlaps)) + 1

In [8]:
assert no_overlaps(test1) == 3

In [9]:
print("Part 2:", no_overlaps(rectangles))

Part 2: 632
